# Exploration des Données pour l'ETF Personnalisé

Ce notebook réalise une exploration des données financières pour la construction d'un ETF personnalisé ciblant les marchés émergents et les technologies.

## Objectifs
- Explorer les données de marché des actifs potentiels pour l'ETF
- Analyser les caractéristiques fondamentales des actifs
- Examiner les métriques de liquidité
- Visualiser les corrélations entre actifs

In [ ]:
# Imports nécessaires
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
from pathlib import Path
import yaml
import json
from datetime import datetime, timedelta

# Configuration pour les visualisations
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("viridis")

# Définir les chemins de données
data_dir = Path("../data")
raw_dir = data_dir / "raw"
processed_dir = data_dir / "processed"
config_path = Path("../config/config.yaml")

# Créer les répertoires s'ils n'existent pas
raw_dir.mkdir(parents=True, exist_ok=True)
processed_dir.mkdir(parents=True, exist_ok=True)

## Chargement de la Configuration

Commençons par charger la configuration du projet depuis le fichier YAML.

In [ ]:
# Charger la configuration
try:
    with open(config_path, 'r') as file:
        config = yaml.safe_load(file)
        print("Configuration chargée avec succès!")
except Exception as e:
    print(f"Erreur lors du chargement de la configuration: {e}")
    # Créer une configuration par défaut si le fichier n'existe pas
    config = {
        "etf": {
            "name": "Innovation Tech Émergente ETF",
            "benchmark": "MSCI Emerging Markets Technology Index"
        },
        "asset_selection": {
            "markets": ["Brazil", "Russia", "India", "China", "South Africa"],
            "sectors": ["Technology", "Telecommunications", "Consumer Electronics"]
        },
        "backtesting": {
            "start_date": "2018-01-01",
            "end_date": "2022-12-31"
        }
    }

# Afficher les informations de base sur l'ETF
print(f"Nom de l'ETF: {config['etf']['name']}")
print(f"Indice de référence: {config['etf']['benchmark']}")
print(f"\nMarchés ciblés: {', '.join(config['asset_selection']['markets'])}")
print(f"Secteurs ciblés: {', '.join(config['asset_selection']['sectors'])}")
print(f"\nPériode de backtest: {config['backtesting']['start_date']} à {config['backtesting']['end_date']}")

## Collecte des Données de Marché

Pour notre ETF, nous avons besoin de données de marché pour les actifs des pays émergents ciblés. Commençons par identifier les ETFs les plus connus pour ces marchés afin d'extraire leurs composants.

In [ ]:
# Définir les ETFs représentatifs pour chaque marché
market_etfs = {
    "Brazil": "EWZ",
    "Russia": "RSX",
    "India": "INDA",
    "China": "MCHI",
    "South Africa": "EZA",
    "Mexico": "EWW",
    "Indonesia": "EIDO",
    "Turkey": "TUR"
}

# Récupérer des informations sur ces ETFs
for market, etf_ticker in market_etfs.items():
    try:
        etf = yf.Ticker(etf_ticker)
        info = etf.info
        print(f"{market} - {etf_ticker}: {info.get('longName', 'N/A')}")
        print(f"  Actifs sous gestion: {info.get('totalAssets', 'N/A'):,}")
        print(f"  Description: {info.get('longBusinessSummary', 'N/A')[:150]}...")
        print("\n")
    except Exception as e:
        print(f"Erreur lors de la récupération des informations pour {etf_ticker}: {e}")

## Analyse des Composants des ETFs Existants

Examinons la composition des ETFs pour identifier les entreprises potentielles pour notre ETF personnalisé, en se concentrant sur les secteurs technologiques.

In [ ]:
# Créer un dictionnaire pour stocker les composants par marché
market_components = {}

# Boucler sur chaque ETF de marché
for market, etf_ticker in market_etfs.items():
    try:
        print(f"Récupération des composants pour {market} ({etf_ticker})...")
        etf = yf.Ticker(etf_ticker)
        holdings = etf.get_holdings()
        
        if holdings is not None and not holdings.empty:
            # Filtrer les secteurs technologiques
            tech_keywords = ['tech', 'software', 'hardware', 'semiconductor', 'telecom', 
                           'communication', 'electronics', 'internet', 'media', 'digital']
            
            tech_holdings = holdings[holdings['Sector'].str.lower().str.contains('|'.join(tech_keywords), na=False)]
            
            print(f"  Nombre total de composants: {len(holdings)}")
            print(f"  Nombre de composants tech: {len(tech_holdings)}")
            
            # Afficher les 5 plus grandes entreprises tech
            if not tech_holdings.empty:
                print("\n  Top 5 entreprises tech par poids:")
                top5 = tech_holdings.sort_values('% of ETF', ascending=False).head(5)
                for idx, row in top5.iterrows():
                    print(f"    {row['Name']} ({row['Ticker']}) - {row['Sector']} - {row['% of ETF']:.2%}")
                
                # Stocker les composants tech
                market_components[market] = tech_holdings
            else:
                print("  Aucune entreprise tech trouvée dans cet ETF.")
        else:
            print(f"  Pas de données de holdings disponibles pour {etf_ticker}")
        
        print("\n")
    except Exception as e:
        print(f"Erreur lors de la récupération des holdings pour {etf_ticker}: {e}\n")

## Exploration des Prix Historiques

Récupérons les prix historiques pour quelques-unes des entreprises technologiques les plus importantes de chaque marché.

In [ ]:
# Créer une liste des tickers technologiques les plus importants
top_tech_tickers = []

for market, holdings in market_components.items():
    if not holdings.empty:
        # Prendre les 3 plus grands tickers tech par marché
        top3 = holdings.sort_values('% of ETF', ascending=False).head(3)
        for ticker in top3['Ticker']:
            if ticker not in top_tech_tickers and pd.notna(ticker):
                top_tech_tickers.append(ticker)

print(f"Nombre de tickers tech sélectionnés: {len(top_tech_tickers)}")
print(f"Tickers: {', '.join(top_tech_tickers)}")

# Récupérer les prix historiques
start_date = config['backtesting']['start_date']
end_date = config['backtesting']['end_date']

print(f"\nRécupération des prix historiques du {start_date} au {end_date}...")

try:
    # Télécharger les prix
    prices_df = yf.download(top_tech_tickers, start=start_date, end=end_date, auto_adjust=True)['Close']
    
    # Afficher les informations sur les données de prix
    print(f"Données de prix récupérées: {prices_df.shape[0]} jours pour {prices_df.shape[1]} tickers")
    
    # Sauvegarder les données de prix
    prices_path = raw_dir / "selected_prices.parquet"
    prices_df.to_parquet(prices_path)
    print(f"Données de prix sauvegardées dans {prices_path}")
except Exception as e:
    print(f"Erreur lors de la récupération des prix: {e}")
    prices_df = pd.DataFrame()